# Setup

In [1]:
import os
import gc
import time
import warnings
import subprocess

gc.enable()
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
pd.set_option('precision', 4)
np.set_printoptions(precision=4)

import xgboost
import optuna
optuna.logging.set_verbosity(optuna.logging.ERROR)

from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner
from optuna.integration import XGBoostPruningCallback
from xgboost import XGBRegressor

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_percentage_error

SEED = 23
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [2]:
#remove cell to run future versions
assert optuna.__version__ == '2.10.1', f'Change in Optuna version. Original notebook version: 2.10.1'
assert xgboost.__version__ == '1.6.1', f'Change in XGBoost version. Original notebook version: 1.6.1'

In [3]:
#Check GPU availability
try:
    subprocess.check_output('nvidia-smi')
    GPU = True
except Exception:
    GPU = False

print(f'GPU available: {GPU}')

GPU available: True


# Data 

In [4]:
data_path = '../input/mh-renew-power-hackathon-data/data/'

train = pd.read_csv(data_path + 'processed/train.csv')
test = pd.read_csv(data_path + 'processed/test.csv')

target = train.pop('Target')

In [5]:
train['turbine_id'] = train['turbine_id'].apply(lambda x: x[8:]).astype('int8')
test['turbine_id'] = test['turbine_id'].apply(lambda x: x[8:]).astype('int8')

In [6]:
train_pca_95 = pd.read_csv(data_path + 'processed/train_pca_95.csv')
train_pca_95['turbine_id'] = train['turbine_id']
test_pca_95 = pd.read_csv(data_path + 'processed/test_pca_95.csv')
test_pca_95['turbine_id'] = test['turbine_id']

train_pca_mle = pd.read_csv(data_path + 'processed/train_pca_mle.csv')
train_pca_mle['turbine_id'] = train['turbine_id']
test_pca_mle = pd.read_csv(data_path + 'processed/test_pca_mle.csv')
test_pca_mle['turbine_id'] = test['turbine_id']

In [7]:
features = [f for f in train.columns if f not in ('timestamp', 'Target')]
cat_features = ['turbine_id']
num_features = [f for f in features if f not in cat_features]

**Feature sets from [EDA notebook](https://github.com/stiwari-ds/data-science-competitions/blob/main/machinehack/renew_power_hackathon/notebooks/01_eda.ipynb)**

In [8]:
vif_features = ['turbine_id', 'grid_power10min_average', 'ambient_temperature', 
                'wind_direction_raw_X', 'wind_direction_raw_Y']

mi_features = ['turbine_id', 'ambient_temperature', 'nacelle_temp', 
               'nc1_inside_temp', 'generator_winding_temp_max', 
               'wind_direction_raw', 'reactive_power']

#by pearson correlation
redundant_features = ['active_power_calculated_by_converter', 'active_power_raw',
                      'wind_speed_raw', 'reactive_power']
non_collinear_features = [f for f in features if f not in redundant_features]

# Hyperparameter tuning

In [9]:
def objective(trial, data, base_params):

    scores = []
    X, y = data
    # X['turbine_id'] = X['turbine_id'].astype('category')

    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.4, step=0.005),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.005),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.005),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0, step=0.005),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.5, 1.0, step=0.005),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 15),
        'gamma': trial.suggest_float('gamma', 0, 20, step=0.005),
        'alpha': trial.suggest_float('alpha', 1e-5, 1e3, log=True),
        'lambda': trial.suggest_float('lambda', 1e-5, 1e3, log=True),
        'max_cat_to_onehot': trial.suggest_categorical('max_cat_to_onehot', [2, 16])
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, X['turbine_id'])):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        model = XGBRegressor(
            **base_params, 
            **param_grid,
            callbacks=[XGBoostPruningCallback(
                trial=trial, 
                observation_key='validation_0-mape')]
        )
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=0
        )
        preds = model.predict(X_val)
        scores.append(mean_absolute_percentage_error(y_val, preds))
    
    return np.mean(scores)

In [10]:
def tune_params(data, base_params, n_trials=10, direction='maximize'):
    study = optuna.create_study(
        sampler=TPESampler(seed=SEED),
        pruner=HyperbandPruner(),
        direction=direction
    )
    
    study.optimize(
        func=lambda trial: objective(trial, data, base_params),
        n_trials=n_trials,
        gc_after_trial=True
    )
    
    return study

# Cross-validation and experiment setup

In [11]:
def evaluate_model(data, model_params):
    
    preds_oof = {} #out-of-fold predictions for train set
    preds_test = []
    scores_mape = [] #validation set MAPE
    
    X, X_test, y = data
    # X['turbine_id'] = X['turbine_id'].astype('category')
    # X_test['turbine_id'] = X_test['turbine_id'].astype('category')
    
    cv_start = time.time()
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, X['turbine_id'])):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        fold_start = time.time()
        model = XGBRegressor(**model_params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=0
        )
        
        preds_val = model.predict(X_val)
        preds_oof.update(dict(zip(val_idx, preds_val)))
        score = mean_absolute_percentage_error(y_val, preds_val)
        scores_mape.append(score)
        preds_test.append(model.predict(X_test))

        fold_end = time.time()
        print(f'Fold #{fold}: {model.best_iteration} rounds, ' \
              f'MAPE = {score:.5f} [Time: {fold_end - fold_start:.2f}s]')
        _ = gc.collect()
    
    cv_end = time.time()
    print(f'Average MAPE = {np.mean(scores_mape):.5f}' \
          f' (with std = {np.std(scores_mape):.5f})')
    print(f'[Total time: {cv_end - cv_start:.2f}s]\n')
    
    preds_oof = pd.DataFrame.from_dict(preds_oof, orient='index').reset_index()
    preds_test = np.mean(np.column_stack(preds_test), axis=1)
    return preds_oof, preds_test

In [12]:
def run_experiment(data, n_trials=5):
        
    X, X_test, y = data
    
    base_params = {
        'objective': 'reg:squarederror',
        'n_estimators': 20000,
        'booster': 'gbtree',
        'eval_metric': 'mape',
        'early_stopping_rounds': 100,
        'tree_method': 'gpu_hist' if GPU else 'hist', 
        'enable_categorical': GPU,
        'predictor': 'gpu_predictor' if GPU else 'cpu_predictor',
#         'max_cat_to_onehot': 16, #internal one-hot encoding for turbine_id
        'verbosity': 1,
        'seed': SEED
    }
    
    print(f'---------------Hyperparameter tuning---------------')
    study = tune_params(
        data=(X, y), 
        base_params=base_params,
        n_trials=n_trials,
        direction='minimize'
    )
    print(f'Best trial: {study.best_trial.number} -> Best value(MAPE): {study.best_value:.5f}')
    print(f'Best hyperparameters:')
    for k, v in study.best_params.items():
        print(f'{k:20} - {v}')
    
    model_params = {**base_params, **study.best_params}
    print(f'-----------------Cross-validation------------------')
    preds_oof, preds_test = evaluate_model(
        data=(X, X_test, y), 
        model_params=model_params
    )
    return preds_oof, preds_test

# Experiments

### All features

In [13]:
%%time
oof_all, preds_all = run_experiment((train[features], test[features], target), 250)

---------------Hyperparameter tuning---------------
Best trial: 203 -> Best value(MAPE): 0.01567
Best hyperparameters:
learning_rate        - 0.19
max_depth            - 15
subsample            - 0.78
colsample_bytree     - 0.88
colsample_bylevel    - 0.505
colsample_bynode     - 0.845
min_child_weight     - 11
gamma                - 0.48
alpha                - 1.0754111739760395e-05
lambda               - 1.1405369999290365e-05
max_cat_to_onehot    - 16
-----------------Cross-validation------------------
Fold #0: 797 rounds, MAPE = 0.01578 [Time: 76.76s]
Fold #1: 627 rounds, MAPE = 0.01561 [Time: 72.59s]
Fold #2: 1184 rounds, MAPE = 0.01560 [Time: 80.26s]
Fold #3: 948 rounds, MAPE = 0.01563 [Time: 77.61s]
Fold #4: 1110 rounds, MAPE = 0.01571 [Time: 77.31s]
Average MAPE = 0.01567 (with std = 0.00007)
[Total time: 385.96s]

CPU times: user 51min 42s, sys: 42.8 s, total: 52min 24s
Wall time: 50min 16s


In [14]:
_ = gc.collect()

In [15]:
submission_path = './'

sub_all = pd.DataFrame({'Target': preds_all})
sub_all.to_csv(submission_path + '02_sub_all.csv', index=False)

### VIF-approved features

In [16]:
%%time
oof_vif, preds_vif = run_experiment((train[vif_features], test[vif_features], target), 250)

---------------Hyperparameter tuning---------------
Best trial: 190 -> Best value(MAPE): 0.02389
Best hyperparameters:
learning_rate        - 0.20500000000000002
max_depth            - 19
subsample            - 0.8500000000000001
colsample_bytree     - 0.895
colsample_bylevel    - 0.75
colsample_bynode     - 0.515
min_child_weight     - 8
gamma                - 1.44
alpha                - 1.509600232848986
lambda               - 0.0003281616316277161
max_cat_to_onehot    - 2
-----------------Cross-validation------------------
Fold #0: 353 rounds, MAPE = 0.02386 [Time: 20.77s]
Fold #1: 578 rounds, MAPE = 0.02391 [Time: 25.65s]
Fold #2: 567 rounds, MAPE = 0.02391 [Time: 24.28s]
Fold #3: 578 rounds, MAPE = 0.02383 [Time: 24.14s]
Fold #4: 638 rounds, MAPE = 0.02396 [Time: 24.98s]
Average MAPE = 0.02389 (with std = 0.00004)
[Total time: 120.96s]

CPU times: user 14min 59s, sys: 6.11 s, total: 15min 5s
Wall time: 14min 27s


In [17]:
_ = gc.collect()

In [18]:
sub_vif = pd.DataFrame({'Target': preds_vif})
sub_vif.to_csv(submission_path + '02_sub_vif.csv', index=False)

### Mutual Information-based features

In [19]:
%%time
oof_mi, preds_mi = run_experiment((train[mi_features], test[mi_features], target), 250)

---------------Hyperparameter tuning---------------
Best trial: 128 -> Best value(MAPE): 0.01656
Best hyperparameters:
learning_rate        - 0.20500000000000002
max_depth            - 19
subsample            - 0.8500000000000001
colsample_bytree     - 0.895
colsample_bylevel    - 0.75
colsample_bynode     - 0.515
min_child_weight     - 8
gamma                - 1.44
alpha                - 1.509600232848986
lambda               - 0.0003281616316277161
max_cat_to_onehot    - 2
-----------------Cross-validation------------------
Fold #0: 1667 rounds, MAPE = 0.01665 [Time: 33.97s]
Fold #1: 562 rounds, MAPE = 0.01662 [Time: 26.45s]
Fold #2: 1918 rounds, MAPE = 0.01648 [Time: 37.03s]
Fold #3: 1226 rounds, MAPE = 0.01655 [Time: 31.09s]
Fold #4: 1413 rounds, MAPE = 0.01653 [Time: 33.01s]
Average MAPE = 0.01656 (with std = 0.00006)
[Total time: 162.74s]

CPU times: user 18min 58s, sys: 20.3 s, total: 19min 19s
Wall time: 18min 27s


In [20]:
_ = gc.collect()

In [21]:
sub_mi = pd.DataFrame({'Target': preds_mi})
sub_mi.to_csv(submission_path + '02_sub_mi.csv', index=False)

### Non-collinear features (Pearson correlation)

In [22]:
%%time
oof_ncol, preds_ncol = run_experiment((train[non_collinear_features], test[non_collinear_features], target), 250)

---------------Hyperparameter tuning---------------
Best trial: 4 -> Best value(MAPE): 0.01592
Best hyperparameters:
learning_rate        - 0.245
max_depth            - 20
subsample            - 0.9550000000000001
colsample_bytree     - 0.755
colsample_bylevel    - 0.91
colsample_bynode     - 0.915
min_child_weight     - 5
gamma                - 1.58
alpha                - 0.05322349770683999
lambda               - 107.59856199744125
max_cat_to_onehot    - 16
-----------------Cross-validation------------------
Fold #0: 574 rounds, MAPE = 0.01599 [Time: 40.56s]
Fold #1: 414 rounds, MAPE = 0.01578 [Time: 38.08s]
Fold #2: 448 rounds, MAPE = 0.01589 [Time: 37.46s]
Fold #3: 610 rounds, MAPE = 0.01602 [Time: 40.25s]
Fold #4: 437 rounds, MAPE = 0.01590 [Time: 36.70s]
Average MAPE = 0.01592 (with std = 0.00009)
[Total time: 194.45s]

CPU times: user 24min 6s, sys: 28.3 s, total: 24min 34s
Wall time: 23min 19s


In [23]:
_ = gc.collect()

In [24]:
sub_ncol = pd.DataFrame({'Target': preds_ncol})
sub_ncol.to_csv(submission_path + '02_sub_ncol.csv', index=False)

### PCA - 95% components

In [25]:
%%time
oof_pca95, preds_pca95 = run_experiment((train_pca_95, test_pca_95, target), 250)

---------------Hyperparameter tuning---------------
Best trial: 197 -> Best value(MAPE): 0.01859
Best hyperparameters:
learning_rate        - 0.20500000000000002
max_depth            - 19
subsample            - 0.8500000000000001
colsample_bytree     - 0.895
colsample_bylevel    - 0.75
colsample_bynode     - 0.515
min_child_weight     - 8
gamma                - 1.44
alpha                - 1.509600232848986
lambda               - 0.0003281616316277161
max_cat_to_onehot    - 2
-----------------Cross-validation------------------
Fold #0: 1014 rounds, MAPE = 0.01859 [Time: 39.38s]
Fold #1: 763 rounds, MAPE = 0.01856 [Time: 36.91s]
Fold #2: 358 rounds, MAPE = 0.01861 [Time: 33.38s]
Fold #3: 643 rounds, MAPE = 0.01858 [Time: 36.18s]
Fold #4: 316 rounds, MAPE = 0.01863 [Time: 33.08s]
Average MAPE = 0.01859 (with std = 0.00002)
[Total time: 180.17s]

CPU times: user 31min 10s, sys: 23.3 s, total: 31min 34s
Wall time: 30min 33s


In [26]:
_ = gc.collect()

In [27]:
sub_pca95 = pd.DataFrame({'Target': preds_pca95})
sub_pca95.to_csv(submission_path + '02_sub_pca95.csv', index=False)

### PCA - MLE components

In [28]:
%%time
oof_pcamle, preds_pcamle = run_experiment((train_pca_mle, test_pca_mle, target), 250)

---------------Hyperparameter tuning---------------
Best trial: 166 -> Best value(MAPE): 0.01782
Best hyperparameters:
learning_rate        - 0.20500000000000002
max_depth            - 19
subsample            - 0.8500000000000001
colsample_bytree     - 0.895
colsample_bylevel    - 0.75
colsample_bynode     - 0.515
min_child_weight     - 8
gamma                - 1.44
alpha                - 1.509600232848986
lambda               - 0.0003281616316277161
max_cat_to_onehot    - 2
-----------------Cross-validation------------------
Fold #0: 631 rounds, MAPE = 0.01781 [Time: 41.79s]
Fold #1: 1053 rounds, MAPE = 0.01784 [Time: 45.86s]
Fold #2: 1159 rounds, MAPE = 0.01775 [Time: 46.94s]
Fold #3: 995 rounds, MAPE = 0.01787 [Time: 45.49s]
Fold #4: 1312 rounds, MAPE = 0.01782 [Time: 48.58s]
Average MAPE = 0.01782 (with std = 0.00004)
[Total time: 230.07s]

CPU times: user 41min 44s, sys: 39.5 s, total: 42min 23s
Wall time: 40min 13s


In [29]:
_ = gc.collect()

In [30]:
sub_pcamle = pd.DataFrame({'Target': preds_pcamle})
sub_pcamle.to_csv(submission_path + '02_sub_pcamle.csv', index=False)

# Storing out-of-fold predictions

In [48]:
oof_all.head()

,index,0
0,3,46.5553
1,9,47.4729
2,16,45.5592
3,23,45.7692
4,24,46.3586


In [52]:
oof_all.rename({0: '02_all'}, axis=1, inplace=True)
oof_all.to_csv(submission_path + '02_oof_all.csv', index=False)

oof_vif.rename({0: '02_vif'}, axis=1, inplace=True)
oof_vif.to_csv(submission_path + '02_oof_vif.csv', index=False)

oof_mi.rename({0: '02_mi'}, axis=1, inplace=True)
oof_mi.to_csv(submission_path + '02_oof_mi.csv', index=False)

oof_ncol.rename({0: '02_ncol'}, axis=1, inplace=True)
oof_ncol.to_csv(submission_path + '02_oof_ncol.csv', index=False)

oof_pca95.rename({0: '02_pca95'}, axis=1, inplace=True)
oof_pca95.to_csv(submission_path + '02_oof_pca95.csv', index=False)

oof_pcamle.rename({0: '02_pcamle'}, axis=1, inplace=True)
oof_pcamle.to_csv(submission_path + '02_oof_pcamle.csv', index=False)

In [53]:
!head 02_oof_mi.csv

index,02_mi
3,46.67413
9,47.24785
16,46.148422
23,45.69135
24,47.339943
28,45.364887
31,48.381268
32,44.390003
34,46.86816
